<font size=16>Lesson 07: Constraint Programming</font>

# Introduction

Many discrete or combinatorial optimization problems don't exactly fit the integer programming paradigm with linear constraints and and a linear objective function as we'll see with several examples below.  Moreover, the constraints can be both many and complicated.  Constraint programming concentrate more on feasibility than on optimization and can deal with many different constraints that aren't available in standard integer programming.

You should read the introductory material on pages 525-531.  In particular we'll use both the element and all-different constraints discussed in the text which are both unlike constraints we've seen before.  For programming we'll be using CP-SAT (Constraint Programming - SATisfiability) by Google.  It's written in C++, but has an API that is part of the or-tools package which you can install using `pip install ortools` (there doesn't appear to be a conda install option).  

We'll start by looking at the example introduced on page 526.  We'll use it to introduce several ideas.

Note, this week we've switched to Python f-strings (formatted string literals) for formatting our output.  They're really slick and the latest innovation in Python text formatting.  <a href="https://realpython.com/python-f-strings/">This is a nice article about f-strings.</a>

# Textbook Example 1 (page 526)

We'll look at the example below and see how it gets solved in with CP-SAT with Python.

Find a feasible solution with the following constraints:

$x_1 \in \left\{ 1, 2 \right\}$ 

$x_2 \in \left\{ 1, 2 \right\}$ 

$x_3 \in \left\{ 1, 2, 3 \right\}$

$x_4 \in \left\{ 1, 2, 3, 4, 5 \right\}$ 

$x_1 + x_3 = 4$ 

$x_1, x_2, x_3, x_4$ are all different.

Note, we aren't trying to optimize anything here.  Instead we're just focused on finding a feasible solution.  Also, notice that the last constraint requiring the 4 variables to have different values is something we haven't seen before.  It could be done in integer programming by using a bunch of inequalities, but the elegance of constraint programming is that can often make it quite simple to declare complicated constraints.

The all-different constraint is an example of a constraint that would be difficult in an integer program, but is one line in a constraint program as we'll see below:

## CP-SAT solution

In [1]:
from ortools.sat.python import cp_model

# Create the model
model = cp_model.CpModel()

# Create the variables
x1 = model.NewIntVar(1,2,'x1')
x2 = model.NewIntVar(1,2,'x2')
x3 = model.NewIntVar(1,3,'x3')
x4 = model.NewIntVar(1,5,'x4')

# Create the constraints
model.Add(x1 + x3 == 4)
model.AddAllDifferent([x1,x2,x3,x4])

# Create a solver and solve the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

# the solver stops when it finds a feasible solution
if status == cp_model.FEASIBLE:
    print('A feasible solution is:')
    for x in [x1,x2,x3,x4]:
        print(f'{x} = {solver.Value(x)}')

A feasible solution is:
x1 = 1
x2 = 2
x3 = 3
x4 = 4


### <font color = "blue"> Self Assessment: Find a feasible solution.</font>

Use CP-SAT to find a feasible solution to 

$x \neq y$

For $x,y,z \in \{0,1,2\}.$

## Finding all feasible solutions

Sometimes we need to collect all of the feasible solutions.  This can be done by telling the solver to search for all of the solutions and then using a callback method to print or record them.  We'll just print them.  First we create a solution printer class that contains a callback method which gets executed by the solver as it enumerates the solutions.  In this case it just updates a counter and prints all of values of the variables we specify.  You can use this without change for other problems.

In [2]:
from ortools.sat.python import cp_model

class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print(f'{v} = {self.Value(v)}', end = ' ')
        print()

    def solution_count(self):
        return self.__solution_count

The code below is mostly a repeat of the code above, but everything beginning with `solution_printer` is new (line 17).  Notice we initialize the solution printer with a list of the variables that we want to print as CP-SAT finds each solution.

In [3]:
from ortools.sat.python import cp_model

# Creates the model.
model = cp_model.CpModel()

x1 = model.NewIntVar(1,2,'x1')
x2 = model.NewIntVar(1,2,'x2')
x3 = model.NewIntVar(1,3,'x3')
x4 = model.NewIntVar(1,5,'x4')

# Creates the constraints.
model.Add(x1 + x3 == 4)
model.AddAllDifferent([x1,x2,x3,x4])

# Creates a solver and solves the model.
solver = cp_model.CpSolver()  # it wasn't really necessary to redo any of the above code
solution_printer = VarArraySolutionPrinter([x1,x2,x3,x4])
status = solver.SearchForAllSolutions(model, solution_printer)

print(f'Status = {solver.StatusName(status)}')
print(f'Number of solutions found: {solution_printer.solution_count()}')

x1 = 1 x2 = 2 x3 = 3 x4 = 4 
x1 = 1 x2 = 2 x3 = 3 x4 = 5 
Status = OPTIMAL
Number of solutions found: 2


###  <font color = "blue"> Self Assessment: Finding all feasible solutions. </font>

Use CP-SAT to print out all of the feasible solutions to 

$x \neq y$

For $x,y,z \in \{0,1,2\}.$

## Finding an optimal value

Even though the focus of constraint programming is finding feasible solutions it can find optimal values of an objective function by comparing the values of the objective function for all feasible solutions.  This can be a slow process if there are too many feasible solutions.  There are a variety of ways to do an incomplete search.  If you're curious to learn more have a look at <a href="https://ktiml.mff.cuni.cz/~bartak/podminky/lectures/CSP-lecture09eng.pdf">these lecture slides</a> as a starting point.

As an example we'll add an objective to the problem we developed above:


Minimize $ 10 x_1 - 2 x_2 + 20 x_3 - x_4$

Subject to:

$x_1 \in \left\{ 1, 2 \right\}, x_2 \in \left\{ 1, 2 \right\}, x_3 \in \left\{ 1, 2, 3 \right\}, x_4 \in \left\{ 1, 2, 3, 4, 5 \right\}$ 

$x_1 + x_3 = 4$ 

$x_1, x_2, x_3, x_4$ are all different.

Adding the code is pretty simple as we just have to add the objective and an extra line to display its optimized value:

In [4]:
from ortools.sat.python import cp_model

# Create the model
model = cp_model.CpModel()

# Create the variables
x1 = model.NewIntVar(1,2,'x1')
x2 = model.NewIntVar(1,2,'x2')
x3 = model.NewIntVar(1,3,'x3')
x4 = model.NewIntVar(1,5,'x4')

# Create the constraints
model.Add(x1 + x3 == 4)
model.AddAllDifferent([x1,x2,x3,x4])

# Add a linear objective function
model.Minimize(10*x1 - 2*x2 + 20*x3 - x4)

# Create a solver and solve the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

# the solver stops when it finds a feasible solution
if status == cp_model.OPTIMAL:
    print(f'The minimum value of the objective function is {solver.ObjectiveValue()}')
    print()
    for x in [x1,x2,x3,x4]:
        print(f'{x} = {solver.Value(x)}')

The minimum value of the objective function is 61.0

x1 = 1
x2 = 2
x3 = 3
x4 = 5


### <font color = "blue"> Self Assessment: Optimizing a linear objective function with CP-SAT </font>

Use CP-SAT to

Maximize $x + 2y + 3z$ 

Subject to:

$x \neq y$

For $x,y,z \in \{0,1,2\}.$

## Using a list of variables to abstract and generalize

We can create lists of variables using list comprehensions to make it easier to solve large problems.  Here is how we could setup the problem above using generalizable code.

Use CP-SAT and lists write generalizable code to solve:

Minimize $ 10 x_1 - 2 x_2 + 20 x_3 - x_4$

Subject to:

$x_1 \in \left\{ 1, 2 \right\}, x_2 \in \left\{ 1, 2 \right\}, x_3 \in \left\{ 1, 2, 3 \right\}, x_4 \in \left\{ 1, 2, 3, 4, 5 \right\}$ 

$x_1 + x_3 = 4$ 

$x_1, x_2, x_3, x_4$ are all different.

Here we create a list of variables and frame the constraints and objective function using indices to access the variables in the list:

In [5]:
from ortools.sat.python import cp_model

"""Minimal CP-SAT example to showcase calling the solver."""
# Creates the model.
model = cp_model.CpModel()

# Creates the variables.
bounds = [[1,2],[1,2],[1,3],[1,5]]
num_vars = 4
x = [model.NewIntVar(bounds[i][0], bounds[i][1], f'x{i}') for i in range(num_vars)]
# note we now have x[0],...,x[3] instead of x1,...,x4

# Creates the constraints.
model.Add(x[0]+x[2]==4)
model.AddAllDifferent(x)

# Add an objective function and a direction, need not be linear
coef = [10,-2,20,-1]
model.Minimize(sum(coef[i]*x[i] for i in range(num_vars)))

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Minimum of objective function: %i' % solver.ObjectiveValue())
    print()
    for i in range(num_vars):
        print(f'x{i} = {solver.Value(x[i])}')

Minimum of objective function: 61

x0 = 1
x1 = 2
x2 = 3
x3 = 5


### <font color = "blue"> Self Assessment: Generalizable code with CP-SAT </font>

Write code that can be easily extended to a larger problem to solve:

Maximize $x + 2y + 3z$ 

Subject to:

$x \neq y$

For $x,y,z \in \{0,1,2\}.$

# A Map Coloring Example

For our purposes this isn't a very important example and we're not going to have you do something like it for the homework, but it's a fun application of constraint programming.  

<img src="./images/europe_countries.png" width=400>

How do we select the colors for regions on a map so that adjacent regions don't have the same colors?  A now famous theorem in mathematics called the <a href="https://en.wikipedia.org/wiki/Four_color_theorem">four color theorem</a> states that we can always do it with no more than 4 colors provided adjacency means that regions touch along an edge not just at a corner.  The theorem is famous because it is the first major theorem to be proved by computer.  The theorem tells us it can be done, but it doesn't tell us how to do it.  It's a relatively easy application of constraint programming to find a feasible coloring.

Here is a concrete version of the code to find a coloring for 6 adjacent European countries:

In [6]:
from ortools.sat.python import cp_model

# Creates the model.
model = cp_model.CpModel()

# Creates the variables.
num_colors = 4
Belgium = model.NewIntVar(0, num_colors - 1, 'Belgium')
Denmark = model.NewIntVar(0, num_colors - 1, 'Denmark')
France = model.NewIntVar(0, num_colors - 1, 'France')
Germany = model.NewIntVar(0, num_colors - 1, 'Germany')
Luxembourg = model.NewIntVar(0, num_colors - 1, 'Luxembourg')
Netherlands = model.NewIntVar(0, num_colors - 1, 'Netherlands')

# Constraints so that adjacent countries get assigned different colors
model.Add(Belgium != France)
model.Add(Belgium != Germany)
model.Add(Belgium != Netherlands)
model.Add(Belgium != Luxembourg)
model.Add(Denmark != Germany)
model.Add(France != Germany)
model.Add(France != Luxembourg)
model.Add(Germany != Luxembourg)
model.Add(Germany != Netherlands)

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.FEASIBLE:
    print(f'Belgium = {solver.Value(Belgium)}')
    print(f'Denmark = {solver.Value(Denmark)}')
    print(f'France = {solver.Value(France)}')
    print(f'Germany = {solver.Value(Germany)}')
    print(f'Luxembourg = {solver.Value(Luxembourg)}')
    print(f'Netherlands = {solver.Value(Netherlands)}')

Belgium = 0
Denmark = 3
France = 3
Germany = 2
Luxembourg = 1
Netherlands = 1


Below we present a version of the solution that can be easily generalized to a large number of countries or regions.  To represent the adjacencies between countries we've used what is called an adjacency matrix where a 1 indicates two countries are adjacent and a 0 indicates they are not.  We can consider countries adjacent to themselves or not, but it doesn't matter since we don't use the information from the diagonal of the matrix.  The adjacencies could also be specified using a list of adjacent pairs, or with a dictionary, or ...

In [7]:
import pandas as pd 
import numpy as np
country_names = ['Belgium','Denmark','France','Germany','Luxembourg','Netherlands']
adjacency_matrix = pd.DataFrame([[1,0,1,1,1,1],
                                 [0,1,0,1,0,0],
                                 [1,0,1,1,1,0],
                                 [1,1,1,1,1,1],
                                 [1,0,1,1,1,0],
                                 [1,0,0,1,0,1]],
                                index=country_names,columns=country_names)

# Creates the model.
model = cp_model.CpModel()

# Creates the variables.
num_colors = 4
countries = [ model.NewIntVar(0, num_colors - 1, c) for c in country_names]

# Creates the constraints from the upper triangular part of the adj. matrix
num_countries = len(countries)
for i in range(num_countries):
    for j in np.arange(i+1,num_countries):
        if adjacency_matrix.iloc[i,j] == 1:
            model.Add( countries[i] != countries[j] )
            
# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print Results
for c in countries:
    print(f'{c} = {solver.Value(c)}')

Belgium = 0
Denmark = 3
France = 3
Germany = 2
Luxembourg = 1
Netherlands = 1


## <font color = "blue"> Self Assessment: How many different ways to color?</font>

Find all the feasible solutions with four colors to determine how many different ways there are to color the map using the same four colors.

# Example - Using Sets of Values

In the example above our variables each took values in a range.  If we instead want to specify a list of possible values then we can use the construction in the following example which uses CP-SAT's concept of domains.

Maximize $x+y$

Subject to: 

$x \in \{ 5, 10, 20\}, y \in \{ 3, 7, 12\}$

$x+y \leq 30$

Here is a concrete implementation of a solution:

In [8]:
from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

# Creates the variables.
x = model.NewIntVarFromDomain(cp_model.Domain.FromValues([5,10,20]), 'x')
y = model.NewIntVarFromDomain(cp_model.Domain.FromValues([3, 7,12]), 'y')

# Creates the constraints.
model.Add(x + y <= 30)

# Add an objective function and a direction, need not be linear
model.Maximize(x + y)

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Maximum of objective function: %i' % solver.ObjectiveValue())
    print()
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))

Maximum of objective function: 27

x = 20
y = 7


## <font color = "blue"> Self Assessment: Using Sets and All-Different </font>

Maximize:  $10 x_1 + 2 x_2 - x_3$

Subject to

$x_1 \in \left\{10, 20, 30\right\}$

$x_2 \in \left\{20, 30, 40\right\}$

$x_3 \in \left\{10, 30, 50\right\}$

$x_1, x_2,$ and $x_3$ are all different. (You'll need the `AddAllDifferent` constraint used in one of the previous examples.)

## <font color = "blue"> Self Assessment: Generalizable Use of Sets </font>

Now solve the same problem as in the previous self assessment, but write generalizable code that could be easily employed to solve a large model.  
You may which to use a dictionary to store the sets for each variable like this:
    
`sets_dict = { 'x1':[10,20,30], 'x2':[20,30,40], 'x3':[10,30,50]}`

You'll have to loop over the dictionary keys (`sets_dict.keys()`) in your list comprehension to declare the variables.

# Example:  Including Nonlinear Terms in the Objective

Suppose we take the previous example and tweak the objective to include a quadratic term so that it now looks like this:

Maximize $x-y^2$

Subject to: 

$x \in \{ 5, 10, 20\}, y \in \{ 3, 7, 12\}$

$x+y \leq 30.$$

If we were to take the code from the previous section and change the objective to:

`model.Maximize(x - y**2)` 

we'd get an error telling us that the objective function contains nonlinear terms!

There's a simple way around this.  Just add an extra variable to represent the quantity $y^2$, say `ysq` and then an additional special constraint that enforces `ysq == y*y`.  Here's the code:

In [9]:
from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

# Creates the variables.
x = model.NewIntVarFromDomain(cp_model.Domain.FromValues([5,10,20]), 'x')
y = model.NewIntVarFromDomain(cp_model.Domain.FromValues([3, 7,12]), 'y')
ysq = model.NewIntVar(9,144,'ysq') # NEW variable to hold y^2, where did that range come from?

# Creates the constraints.
model.Add(x + y <= 30)
model.AddMultiplicationEquality(ysq, [y, y]) # NEW to enforce ysq = y*y
# can be generalized so that target = product of all items in second list

# Add an objective function and a direction, need not be linear
model.Maximize(x - ysq)

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Maximum of objective function: %i' % solver.ObjectiveValue())
    print()
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))

Maximum of objective function: 11

x = 20
y = 3


If you think it through, you'll see that the solution is correct!

## <font color = "blue"> Self Assessment: Add your own quadratic term</font>

This is a tweaked version of the self-assessment from the previous section with the objective function changed to include a quadratic term.

Maximize:  $10 x_1 + 2 x_2 - x_3^2$

Subject to

$x_1 \in \left\{10, 20, 30\right\}$

$x_2 \in \left\{20, 30, 40\right\}$

$x_3 \in \left\{10, 30, 50\right\}$

$x_1, x_2,$ and $x_3$ are all different.

Write code to use CP-SAT to solve this problem.  Concrete code or generalizable code is fine.  Our solution is written concretely, but it wouldn't be hard to create an extra list of variables to contain all the quadratic variables and then use a loop to add all the necessary `MultiplicationEquality` constraints to make generalizable code.

# The Element Constraint and Assignment Problems

The element constraint allows us to use decision variables as indices to arrays.  It is discussed on page 529 and the syntax in CP-SAT is very similar.  Before showing how the element constraint can be used to solve assignment problems we'll show how to use it to pick values from sets as we did earlier for this problem:

Maximize $x+y$

Subject to: 

$x \in \{ 5, 10, 20\}, y \in \{ 3, 7, 12\}$

$x+y \leq 30$.

The previous Python solution is better than this one, but we're including this to illustrate how the element constraint works.  We'll use two main decision variables `x_idx` and `y_idx` whose job is to specify which element of $x$ and $y$ to use.

In [10]:
from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

x_list = [5, 10, 20]
y_list = [3,  7, 12]

x_idx = model.NewIntVar(0,2,'x_idx')
y_idx = model.NewIntVar(0,2,'y_idx')

x = model.NewIntVar(0,20,'x')
y = model.NewIntVar(0,12,'y')

model.AddElement(x_idx, x_list, x) # x = x_list[x_idx]
model.AddElement(y_idx, y_list, y) # y = y_list[y_idx]
model.Add( x + y <= 30)

# Add an objective function and a direction, need not be linear
model.Maximize(x + y)

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print('Maximum of objective function: %i' % solver.ObjectiveValue())
    print()
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))

Maximum of objective function: 27

x = 20
y = 7


## A Concrete CP-SAT Approach to Assignment

Similarly we can use the element constraint to look up costs or profits associated with assigning workers to jobs or machines to locations as in the Job Shop Co. problem described on pages 348-350 of the textbook.  In that problem we have to assign each of 3 machines to one of 4 different locations.  Each machine can only be in one location and machine 2 can't be assigned to location 2.  There is a cost associated with each assignment and we want to minimize the total cost of placing the 3 machines.

*To use the element constraint for an assignment problem you should assign the elements of the smaller set.  In this problem for example, we'll assign 3 machines to 4 locations.  If we were to assign 4 locations to 3 machines, then we'd need an extra "dummy" machine.*

We will have one decision variable for each machine whose job it is to specify the location of the machine.  We'll use the element constraint to look up the costs and we'll use the all-different constraint to make sure the 3 machines are assigned to different locations.  Here are the costs (table is from page 349):

<img src = "./images/job_shop_cost_table.png" width = 400>

We'll use a "bigM" value of 99 to avoid assigning Machine 2 to Location 2.

In [2]:
# concrete version
import pandas as pd
from IPython.display import display, HTML

locations = ['Loc1', 'Loc2', 'Loc3', 'Loc4']
machines = ['Mach1', 'Mach2', 'Mach3']
cost = [ [13, 16, 12, 11],
         [15, 99, 13, 20],
         [ 5,  7, 10,  6]]

num_locations = len(cost[0])

from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

# Creates the variables.
assign0 = model.NewIntVar(0,num_locations-1,'assign0')
assign1 = model.NewIntVar(0,num_locations-1,'assign1')
assign2 = model.NewIntVar(0,num_locations-1,'assign2')

cost0 = model.NewIntVar(0,99,'cost0')
cost1 = model.NewIntVar(0,99,'cost1')
cost2 = model.NewIntVar(0,99,'cost2')


model.AddAllDifferent([assign0,assign1, assign2])
model.AddElement(assign0,cost[0],cost0) # cost0 = cost[0][assign0]
model.AddElement(assign1,cost[1],cost1) # cost1 = cost[1][assign1]
model.AddElement(assign2,cost[2],cost2) # cost2 = cost[2][assign2]

model.Minimize(cost0 + cost1 + cost2)

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print(f'Lowest Possible Cost: {solver.ObjectiveValue()}')
    print()
    print('Assignments and associated costs:')
    cost_assigns = pd.DataFrame(0, index=machines, columns=locations)
    cost_assigns.iloc[ 0, solver.Value(assign0) ] = solver.Value(cost0)
    cost_assigns.iloc[ 1, solver.Value(assign1) ] = solver.Value(cost1)
    cost_assigns.iloc[ 2, solver.Value(assign2) ] = solver.Value(cost2)
    display(cost_assigns)

Lowest Possible Cost: 29.0

Assignments and associated costs:


,Loc1,Loc2,Loc3,Loc4
Mach1,0,0,0,11
Mach2,0,0,13,0
Mach3,5,0,0,0


## An Abstract CP-SAT Approach to Assignment

You can see in the code above that many lines of code are essentially repeated three times.  Imagine if there 100 assignments or more, then the code above would be really gross.  We present an abstracted version of the code below that can be easily generalized to larger or different problems.

In [1]:
# abstract version
import pandas as pd

locations = ['Loc1', 'Loc2', 'Loc3', 'Loc4']
machines = ['Mach1', 'Mach2', 'Mach3']
cost_table = [[13, 16, 12, 11], [15, 99, 13, 20], [5, 7, 10, 6]]

num_locations = len(cost_table[0])
num_machines = len(cost_table)

from ortools.sat.python import cp_model

# Create the model.
model = cp_model.CpModel()

# Variables
assign = [
    model.NewIntVar(0, num_locations - 1, machines[i])
    for i in range(num_machines)
]

#get the maximum cost from the cost_table for the top of our intvar range
max_cost = max(list(map(max, cost_table)))
cost = [model.NewIntVar(0, max_cost, f'cost{i}') for i in range(num_machines)]

# Constraints
model.AddAllDifferent(assign)

for i in range(num_machines):
    model.AddElement(assign[i], cost_table[i], cost[i])

model.Minimize(sum(cost))

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print(f'Lowest Possible Cost: {solver.ObjectiveValue()}')
    print()
    print('Assignments and associated costs:')
    cost_assigns = pd.DataFrame(0, index=machines, columns=locations)
    for i in range(num_machines):
        cost_assigns.iloc[i, solver.Value(assign[i])] = solver.Value(cost[i])
    display(cost_assigns)

Lowest Possible Cost: 29.0

Assignments and associated costs:


,Loc1,Loc2,Loc3,Loc4
Mach1,0,0,0,11
Mach2,0,0,13,0
Mach3,5,0,0,0


## <font color = "blue"> Self Assessment: Textbook Problem 9.3-4 </font>

We first solved this problem in Homework 3:

The coach of an age group swim team needs to assign swimmers to a 200-yard medley relay team to send to the Junior Olympics. Since most of his best swimmers are very fast in more than one stroke, it is not clear which swimmer should be assigned to each of the four strokes. The five fastest swimmers and the best times (in seconds) they have achieved in each of the strokes (for 50 yards) are

<img src = "images/swim.png" width="500">

The coach wishes to determine how to assign four swimmers to the four different strokes to minimize the sum of the corresponding best times.  

Use a generalizable approach constraint programming to solve this problem.  Be sure to identify both the minimum total time and the swimmer assignments.  Note, you'll need to multiply the times by 10 so that they're integers and then divide the total time by 10 to report the result.

## <font color = "blue"> Self Assessment: Ken must swim! </font>

This one is a continuation from the previous problem and is a bit of a challenge to learn something on your own.  

Suppose the coach decides that for the next swim meet Ken will be on the relay even if the total time is larger than before.  Add an extra constraint to the code so that all variable assignments that exclude Ken are not allowed.

Use the `AddForbiddenAssignments` constraint to forbid all assignments that exclude Ken.  You can look it up in the <a href="https://developers.google.com/optimization/reference/python/sat/python/cp_model">reference manual for CP-SAT.</a>  The assignments you need to exclude are those that include only the swimmers 0,1,2, and 3 so you'll need a list of all permutations of those four numbers.  Look up `permutations` in the `itertools` package and remember to convert the `itertools` object to a list of permutations with `list`.

# Scheduling Problems

A common application for constraint programming is to generate feasible schedules of workers to shifts, sports league play schedules, processing jobs to machines, etc.  Often constraint programming tools have special functionality built in to facilitate scheduling problems.  

We'll explore a simple example below, but if you wish to see more examples you could explore the <a href="https://developers.google.com/optimization/scheduling">CP-SAT documention on scheduling problems.</a>

## A Housebuilding Example

We have a housebuilding problem in which there are 10 tasks of fixed size, each of which needs to be assigned a starting time in such a way as to minimize the total time.  Moreover, there are precedence constraints wherein some tasks must be completed before others are started.  The table below summarizes the durations and precedence constraints: 

task | duration | must be done before
---- | ---- | ----
masonry | 35 | carpentry, plumbing, ceiling
carpentry | 15 | roofing
plumbing | 40| facade, garden
ceiling | 15 | painting
roofing | 5 | windows, facade, garden
painting | 10 | moving
windows | 5 | moving
facade | 10 | moving
garden | 5 | moving
moving | 5 |

We'll lay out how to do this with CP-SAT in a very concrete way first, then we'll code it in a way that could be generalized to other problems.

## A concrete CP-SAT solution

First we initialize the model and create variables for the start times of each task.  To make the range of the variables adequately large we initialize them with the total time required if all the tasks were done without overlap.

In [3]:
from ortools.sat.python import cp_model

# Creates the model.
model = cp_model.CpModel()

horizon = 35 + 15 + 40 + 15 + 5 + 10 + 5 + 10 + 5 + 5

start_masonry = model.NewIntVar(0, horizon, 'start_masonry')
start_carpentry = model.NewIntVar(0, horizon, 'start_carpentry')
start_plumbing = model.NewIntVar(0, horizon, 'start_plumbing')
start_ceiling = model.NewIntVar(0, horizon, 'start_ceiling')
start_roofing = model.NewIntVar(0, horizon, 'start_roofing')
start_painting = model.NewIntVar(0, horizon, 'start_painting')
start_windows = model.NewIntVar(0, horizon, 'start_windows')
start_facade = model.NewIntVar(0, horizon, 'start_facade')
start_garden = model.NewIntVar(0, horizon, 'start_garden')
start_moving = model.NewIntVar(0, horizon, 'start_moving')

Now we could use only these variables and build the task durations right into the precedence constraints, it is convenient and better for generalization to introduce variables for the end times of the tasks.

In [4]:
end_masonry = model.NewIntVar(0, horizon, 'end_masonry')
end_carpentry = model.NewIntVar(0, horizon, 'end_carpentry')
end_plumbing = model.NewIntVar(0, horizon, 'end_plumbing')
end_ceiling = model.NewIntVar(0, horizon, 'end_ceiling')
end_roofing = model.NewIntVar(0, horizon, 'end_roofing')
end_painting = model.NewIntVar(0, horizon, 'end_painting')
end_windows = model.NewIntVar(0, horizon, 'end_windows')
end_facade = model.NewIntVar(0, horizon, 'end_facade')
end_garden = model.NewIntVar(0, horizon, 'end_garden')
end_moving = model.NewIntVar(0, horizon, 'end_moving')

The end time minus the start time of task must equal the duration of the task.  This can be handled by adding equality constraints:

In [5]:
model.Add(end_masonry == start_masonry + 35)
model.Add(end_carpentry == start_carpentry + 15)
model.Add(end_plumbing == start_plumbing + 40)
model.Add(end_ceiling == start_ceiling + 15)
model.Add(end_roofing == start_roofing + 5 )
model.Add(end_painting == start_painting + 10)
model.Add(end_windows == start_windows + 5)
model.Add(end_facade == start_facade + 10)
model.Add(end_garden == start_garden + 5)
model.Add(end_moving == start_moving + 5)

When we write this in a more general way we'll introduce interval variables and "no-overlap" constraints.  Now the precedence constraints:

In [6]:
model.Add(end_masonry <= start_carpentry)
model.Add(end_masonry <= start_plumbing)
model.Add(end_masonry <= start_ceiling)
model.Add(end_carpentry <= start_roofing)
model.Add(end_plumbing <= start_facade)
model.Add(end_plumbing <= start_garden)
model.Add(end_ceiling <= start_painting)
model.Add(end_roofing <= start_windows)
model.Add(end_roofing <= start_facade)
model.Add(end_roofing <= start_garden)
model.Add(end_windows <= start_moving)
model.Add(end_facade <= start_moving)
model.Add(end_garden <= start_moving)

The objective is to make the shortest schedule which we can do my minimizing the largest end time.  To do this we create a new objective variable and use the `AddMaxEquality` constraint to set it equal to the maximum of the end times for all of the jobs.  Note that everything we've done to this point in this problem is just integer programming and could be handled by Pyomo, but Pyomo couldn't handle the objective function:

In [7]:
obj_var = model.NewIntVar(0, horizon, 'makespan')
model.AddMaxEquality(obj_var, [
    end_masonry, end_carpentry, end_plumbing, end_ceiling, end_roofing,
    end_painting, end_windows, end_facade, end_garden, end_moving
])
model.Minimize(obj_var)

Solve the model:

In [8]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

Now we can display the total schedule and its length:

In [9]:
print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
print(f'Masonry start at {solver.Value(start_masonry)} and end at {solver.Value(end_masonry)}')
print(f'Carpentry start at {solver.Value(start_carpentry)} and end at {solver.Value(end_carpentry)}')
print(f'Plumbing start at {solver.Value(start_plumbing)} and end at {solver.Value(end_plumbing)}')
print(f'Ceiling start at {solver.Value(start_ceiling)} and end at {solver.Value(end_ceiling)}')
print(f'Roofing start at {solver.Value(start_roofing)} and end at {solver.Value(end_roofing)}')
print(f'Painting start at {solver.Value(start_painting)} and end at {solver.Value(end_painting)}')
print(f'Windows start at {solver.Value(start_windows)} and end at {solver.Value(end_windows)}')
print(f'Facade start at {solver.Value(start_facade)} and end at {solver.Value(end_facade)}')
print(f'Garden start at {solver.Value(start_garden)} and end at {solver.Value(end_garden)}')
print(f'Moving start at {solver.Value(start_moving)} and end at {solver.Value(end_moving)}')

Optimal Schedule Length: 90.0
Masonry start at 0 and end at 35
Carpentry start at 35 and end at 50
Plumbing start at 35 and end at 75
Ceiling start at 35 and end at 50
Roofing start at 50 and end at 55
Painting start at 50 and end at 60
Windows start at 55 and end at 60
Facade start at 75 and end at 85
Garden start at 75 and end at 80
Moving start at 85 and end at 90


If the tasks were done in succession the total time would be 145, but by allowing tasks to overlap where possible we reduced the total time to 90.  However, we really didn't enjoy typing out all of those variables and constraints individually, this problem really should be formulated abstractly for anything more than a few tasks.

## A Generalizable CP-SAT solution

In [10]:
import numpy as np

task_duration_dict = {
    'masonry': 35,
    'carpentry': 15,
    'plumbing': 40,
    'ceiling': 15,
    'roofing': 5,
    'painting': 10,
    'windows': 5,
    'facade': 10,
    'garden': 5,
    'moving': 5
}
task_names = list(task_duration_dict.keys())
num_tasks = len(task_names)
durations = list(task_duration_dict.values())

# for each task we have a list of tasks that must go after
# task:['these','tasks','after']
precedence_dict = {
    'masonry': ['carpentry', 'plumbing', 'ceiling'],
    'carpentry': ['roofing'],
    'plumbing': ['facade', 'garden'],
    'ceiling': ['painting'],
    'roofing': ['windows', 'facade', 'garden'],
    'painting': ['moving'],
    'windows': ['moving'],
    'facade': ['moving'],
    'garden': ['moving']
}

task_name_to_number_dict = dict(zip(task_names, np.arange(0, num_tasks)))

horizon = sum(task_duration_dict.values())

from ortools.sat.python import cp_model
model = cp_model.CpModel()

start_vars = [
    model.NewIntVar(0, horizon, name=f'start_{t}') for t in task_names
]
end_vars = [model.NewIntVar(0, horizon, name=f'end_{t}') for t in task_names]

# the `NewIntervalVar` are both variables and constraints, the internally enforce that start + duration = end
intervals = [
    model.NewIntervalVar(start_vars[i],
                         durations[i],
                         end_vars[i],
                         name=f'interval_{task_names[i]}')
    for i in range(num_tasks)
]

# precedence constraints
for before in list(precedence_dict.keys()):
    for after in precedence_dict[before]:
        before_index = task_name_to_number_dict[before]
        after_index = task_name_to_number_dict[after]
        model.Add(end_vars[before_index] <= start_vars[after_index])

obj_var = model.NewIntVar(0, horizon, 'largest_end_time')
model.AddMaxEquality(obj_var, end_vars)
model.Minimize(obj_var)

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
for i in range(num_tasks):
    print(
        f'{task_names[i]} start at {solver.Value(start_vars[i])} and end at {solver.Value(end_vars[i])}'
    )

Optimal Schedule Length: 90.0
masonry start at 0 and end at 35
carpentry start at 35 and end at 50
plumbing start at 35 and end at 75
ceiling start at 35 and end at 50
roofing start at 50 and end at 55
painting start at 50 and end at 60
windows start at 55 and end at 60
facade start at 75 and end at 85
garden start at 75 and end at 80
moving start at 85 and end at 90


## Visualizing a Schedule

A horizontal bar-like graph called a Gantt chart is often used to visualize scheduled tasks:

In [11]:
# use bokeh to make a Gantt chart
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

output_notebook()
starts = [solver.Value(start_vars[i]) for i in range(num_tasks)]
ends = [solver.Value(end_vars[i]) for i in range(num_tasks)]

source = ColumnDataSource(data=dict(tasks=task_names, starts = starts, ends=ends))

p = figure(x_range=(0,solver.ObjectiveValue()), y_range=task_names, plot_height=350, title="Task Time Spans",
           toolbar_location=None, tools="")

p.hbar(y='tasks', left='starts', right='ends', height=0.9, source=source)

p.xaxis.axis_label = "Time"
p.ygrid.grid_line_color = None

show(p)

Loading BokehJS ...

## <font color = "blue"> Self Assessment: Add a task</font>

Suppose there is an additional housebuilding task to be included.  The "insulation" task has duration 15 and must be done before ceiling and after carpentry, and plumbing.  Add this task and compute the new schedule.  Use the generalizable code to add this task and also make a Gantt chart to visualize the new schedule. *Hopefully this will help convince you of the power of writing generalizable code!*

# Traveling Salesman Problem

This section is here just for fun and you needn't worry about solving a TSP for this lesson.  The Google or-tools package has tools for routing problems that are built on top of CP-SAT.  In particular they solve the TSP problems by a combination of a heuristic first search (like two-opt for instance) followed by some kind of local search.

To learn more visit the or-tools documentation.  We <a href="https://developers.google.com/optimization/routing/tsp">adapted the code from here</a> to solve the 48 capitals TSP we've seen previously.  Run the code below to see it come up with what we think is the optimal solution in about 30 seconds:

In [12]:
"""Simple travelling salesman problem between cities."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# imports
import json

def create_data_model():
    # load data (this may have to be adapted for different problems)
    with open("data/Caps48.json", "r") as tsp_data:
        tsp = json.load(tsp_data)
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = tsp["DistanceMatrix"]
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

def print_solution(manager, routing, assignment):
    """Prints assignment on console."""
    print('Objective: {} km'.format(assignment.ObjectiveValue()/1000))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}kilometers\n'.format(route_distance/1000)

# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

search_parameters.local_search_metaheuristic = (
routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.seconds = 10
search_parameters.log_search = True

# Solve the problem.
assignment = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if assignment:
    print_solution(manager, routing, assignment)

Objective: 17357.978 km
Route for vehicle 0:
 0 -> 7 -> 37 -> 30 -> 43 -> 17 -> 6 -> 27 -> 5 -> 36 -> 18 -> 26 -> 16 -> 42 -> 29 -> 35 -> 45 -> 32 -> 19 -> 46 -> 20 -> 31 -> 38 -> 47 -> 4 -> 41 -> 23 -> 9 -> 44 -> 34 -> 3 -> 25 -> 1 -> 28 -> 33 -> 40 -> 15 -> 21 -> 2 -> 22 -> 13 -> 24 -> 12 -> 10 -> 11 -> 14 -> 39 -> 8 -> 0



That's pretty neat!